In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time


## getting started 




In [3]:
## personal info

from personal import chrome_driver_path, naver_url, craw_url, naver_id, naver_pw

'''personal.py code look like this, fill the variable 
chrome_driver_path = "/chromedriver_win32" 
naver_url = 
craw_url = 
naver_id = 
naver_pw = 
'''


'personal.py code look like this, fill the variable \nchrome_driver_path = "/chromedriver_win32" \nnaver_url = \ncraw_url = \nnaver_id = \nnaver_pw = \n'

In [4]:
## 기본 로그인 basic login
## cccv 로 안막히는 네이버님 감사합니다. 

driver = webdriver.Chrome(chrome_driver_path)
driver.get(naver_url)


login_button = driver.find_element(By.CSS_SELECTOR, "#account > a")
login_button.click()

input_id = driver.find_element(By.CSS_SELECTOR, "#id")
input_pw = driver.find_element(By.CSS_SELECTOR, "#pw")

##input_id.send_keys(naver_id)
##input_pw.send_keys(naver_pw)

input_id.click()
pyperclip.copy(naver_id)
input_id.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

input_pw.click()
pyperclip.copy(naver_pw)
input_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

#browser.execute_script(f"document.getElementsByName('pw')[0].value='{naver_pw}'")
## clipboard ctrl + v accident reduce 
pyperclip.copy("blank")

login_send = driver.find_element(By.CSS_SELECTOR, "#log\.login")
login_send.click()

C:\Users\choi4\AppData\Local\Temp\ipykernel_18284\4161735948.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path)


In [5]:
## 게시판 접근 enter board 

driver.get(craw_url)
view_btn = driver.find_element(By.CSS_SELECTOR,"#menuLink385")
view_btn.click()

In [6]:
## 검색 search 

## 먼저, 네이버 카페의 경우 전체 페이지가 iframe 이므로, iframe > frame 으로 변환하여 내부 CSS 요소를 탐색 
## first, in naver cafe case.entire pages is iframe, convert iframe > frame to find internel CSS elements 

search_blank = "#query"
excute_btn = "#main-area > div.list-search > form > div.input_search_area > button"
item_search = "3080"
mode_sel = "#searchOptionSelectDiv > a"
change_view = "#searchOptionSelectDiv > ul > li:nth-child(1) > a"

## do not did twice (frame change)
i_search = driver.find_element(By.CSS_SELECTOR, "#cafe_main")

driver.switch_to.frame(i_search)
time.sleep(1)





#topLayerQueryInput
#main-area > div.list-search > form > div.input_search_area >
#main-area > div.list-search > form > div.input_search_area > div > query

In [7]:
## inline frame element click 
search = driver.find_element(By.CSS_SELECTOR,search_blank)
time.sleep(1)

search.click()
pyperclip.copy(item_search)
search.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

excute = driver.find_element(By.CSS_SELECTOR,excute_btn)
excute.click()



In [8]:
board_toggle = driver.find_element(By.CSS_SELECTOR, '#searchOptionSelectDiv > a')
toggle_change = driver.find_element(By.CSS_SELECTOR,'#searchOptionSelectDiv > ul > li:nth-child(1) > a')

board_toggle.click()
toggle_change.click()


In [ ]:
import datetime
from openpyxl import Workbook

# 총 몇 페이지 자료를 모을지 선택
total_page = 10

# 페이지 개수 나누기
total_next = total_page // 10
last_page = total_page - total_next * 10

thing = item_search

# datetime
now = datetime.datetime.now()
today = now.strftime('%Y-%m-%d')

# 엑셀
wb = Workbook(write_only=True)
ws = wb.create_sheet(today)
ws.append(['작성날짜', '판매 상태', '제목', 'url', '가격', '내용'])

In [ ]:
## base code https://flight-developer-stroy.tistory.com/6 

# 크롤링 함수 정의(만약 "이전" 버튼이 있으면 num은 1 아니면 0)
def crolling(num):
    # 게시글 들어가는 반복문
    with_before = 0
    for i in range(len(driver.find_elements(By.CSS_SELECTOR,'.article'))):

        # 게시글 들어가기
        articles = driver.find_elements(By.CSS_SELECTOR,'a.article')[i]
        articles.click()
        time.sleep(1)


        # 정보추출
        write_date = driver.find_element(By.CSS_SELECTOR,'span.date').text
        product_title = driver.find_element(By.CSS_SELECTOR,'h3.title_text').text
        url = driver.find_element(By.CSS_SELECTOR,'.button_url').get_attribute('href')
        text_data = driver.find_element(By.CSS_SELECTOR,'#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer > div:nth-child(3) > div.content.CafeViewer > div > div').text
        # 가격을 못찾으면 그냥 빈칸 입력
        try:
            # 가격 문자열을 숫자로 바꾸기
            product_price_str = driver.find_element(By.CSS_SELECTOR,'.ProductPrice').text
            price_no_won = product_price_str[:-1]
            price_no_won_shim = price_no_won.replace(',', '')
            product_price = int(price_no_won_shim)
        except:
            # 제목에서 가격 문자열 추출
            try:
                product_title = product_title.replace('[', '')
                product_title = product_title.replace(']', '&')
                product_price_str = product_title.split('&')[-2]

                # 가격 문자열을 숫자로 바꾸기
                price_no_won = product_price_str[:-1]
                price_no_won_shim = price_no_won.replace(',', '')
                product_price = int(price_no_won_shim)
            except:
                product_price = ''
        # 판매 상태 정보 저장
        try:
            status = driver.find_element(By.CSS_SELECTOR,'.SaleLabel').text
        except:
            status = ""

        # 엑셀에 작성
        ws.append([write_date, status, product_title, url, product_price, text_data])

        # 뒤로가기
        driver.back()
        driver.switch_to.frame('cafe_main')

    # 다음 게시글 page 이동
    pages = driver.find_elements(By.CSS_SELECTOR,'.prev-next a')[page + 1 + num]
    pages.click()

In [ ]:




for i in range(total_next):
    # 마지막 10단위 페이지일 때
    if i == 0:
        # 다음페이지 클릭 반복문
        for page in range(10):
            crolling(0)
    elif i > 0 and i != total_next-1:
        for page in range(10):
            crolling(1)
    elif i == total_next-1:
        for page in range(last_page):
            crolling(1)


# selenium 끝내고 엑셀 파일 저장
# driver.quit() > 끄지마
wb.save(f'중고나라 {today}{thing} 매물.xlsx')


In [ ]:
write_date = driver.find_element(By.CSS_SELECTOR,'span.date').text
    
print(write_date)
    

In [ ]:
text_data = driver.find_element(By.CSS_SELECTOR,'#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer > div:nth-child(3) > div.content.CafeViewer > div > div').text
print(text_data)